# Using pretrained mouse marker on oscope dataset

In [17]:
import sys
base = "./../../../"
sys.path.append(base)

In [18]:
import json
import pandas
import pairs_flat_v2 as pairs
import helper
from sklearn.preprocessing import QuantileTransformer
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import numpy as np

init_notebook_mode(connected=True)

## Loading Mouse-Human Orthologues

In [19]:
genes_mouse_to_human = {}
header = True
count = 0
ambigious = set()

for line in open(base + "data/biomart_mouse-human-orthologs.txt"):
    if header:
        header = False
        continue
    infos = line.split(",")
    if infos[2] != "":
        if infos[0] in genes_mouse_to_human:
            ambigious.add(infos[0])
            count += 1
        genes_mouse_to_human[infos[0]] = infos[2]
            
for gene in ambigious:
    del genes_mouse_to_human[gene]
    
print("{} ambiguous mappings skipped".format(count))
print("{} genes remain".format(len(genes_mouse_to_human)))

6912 ambiguous mappings skipped
18962 genes remain


## Loading Mouse Marker Pairs

In [20]:
marker_json = json.load(
    open(base + 'data/mouse_pretrained-pairs.json')
)

mm_marker_pairs = {
    "G1": list(
        zip(
            marker_json["G1"]["first"],
            marker_json["G1"]["second"]
        )
    ),
    "S": list(
        zip(
            marker_json["S"]["first"],
            marker_json["S"]["second"]
        )
    ),
    "G2M": list(
        zip(
            marker_json["G2M"]["first"],
            marker_json["G2M"]["second"]
        )
    )
}

mm_marker_pairs = {
    phase: [
        (
            genes_mouse_to_human[pair[0]], 
            genes_mouse_to_human[pair[1]]
        ) 
        for pair in pairs 
        if pair[0] in genes_mouse_to_human 
        and pair[1] in genes_mouse_to_human
    ] for phase, pairs in mm_marker_pairs.items()
}

print("The {} marker pairs loaded".format(
          len(mm_marker_pairs["G1"]) +
          len(mm_marker_pairs["S"]) +
          len(mm_marker_pairs["G2M"])
      )
     )
print("Split up into: " 
      "{} G1 , {} S and {} G2M pairs".format(
          len(mm_marker_pairs["G1"]), 
          len(mm_marker_pairs["S"]), 
          len(mm_marker_pairs["G2M"])
      )
     )

The 26545 marker pairs loaded
Split up into: 11259 G1 , 5920 S and 9366 G2M pairs


## Loading Oscope Dataset

In [21]:
gencounts_oscope = pandas.read_csv(
    base + "data/GSE64016_H1andFUCCI_normalized_EC_human.csv"
)
gencounts_oscope.set_index("Unnamed: 0", inplace=True)
gencounts_oscope_sorted = gencounts_oscope.iloc[
    :, [
        gencounts_oscope.columns.get_loc(c) 
        for c in gencounts_oscope.columns if
        "G1_" in c or "G2_" in c or "S_" in c
    ]
]
gencounts_oscope.head(10)

,H1_Exp1.001,H1_Exp1.002,H1_Exp1.003,H1_Exp1.004,H1_Exp1.006,H1_Exp1.007,H1_Exp1.008,H1_Exp1.009,H1_Exp1.010,H1_Exp1.011,...,G1_Exp1.008,G1_Exp1.055,G1_Exp1.050,G1_Exp1.076,G1_Exp1.011,G1_Exp1.063,G1_Exp1.083,G1_Exp1.030,G1_Exp1.018,G1_Exp1.046
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
MKL2,24.148634,285.530829,6.481959,107.129620,0.000000,5.316709,42.809004,0.000000,267.202286,2.838761,...,3.887628,84.337868,69.192927,1.126491,1.137330,0.000000,36.741767,11.218839,152.792860,123.041274
CD109,2.414863,2.238421,341.512799,14.896119,16.807235,115.372585,7.991014,154.389316,16.663439,12.022155,...,4.956726,7.208501,4.299846,7.626347,5.936864,2.797575,149.063512,2.804710,15.996667,7.077119
ABTB1,0.000000,49.351007,0.000000,2.550705,0.000000,23.925190,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
MAST2,0.000000,234.417285,88.586769,0.000000,0.000000,5.316709,0.000000,0.000000,0.000000,11.355046,...,0.971907,15.069427,0.000000,60.830538,22.746604,0.000000,0.720427,0.000000,5.589983,33.230125
KAT5,0.000000,12.443504,114.341752,51.422218,0.000000,16.721050,0.000000,0.000000,151.218876,173.817364,...,0.000000,96.041736,0.000000,0.000000,0.000000,0.000000,1.030210,55.091510,4.826018,0.000000
WWC2,205.118496,8.812680,658.999139,104.706450,374.666774,394.180805,0.000000,219.069622,539.029228,725.303552,...,169.111820,233.122597,656.509085,412.712674,1075.186455,661.889223,104.641994,322.527589,47.514853,67.124853
CD163,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
MYL2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
UBE2Z,43.467541,257.418380,230.606485,22.956347,36.938978,295.077349,23.782780,43.813924,39.639900,102.195412,...,138.010796,58.877380,538.955243,5.632457,71.651801,0.656708,2.881707,610.984967,0.000000,3.215239


## Predicting

In [22]:
prediction = pairs.cyclone(
    gencounts_oscope_sorted, 
    mm_marker_pairs, 
    processes=10,
    verbose=True
)

[__set_matrix] Original Matrix 'x' has shape 19084 x 247
[__set_matrix] Matrix truncation done. Working with 19084 genes for 247 samples.
[cyclone] Preparing marker pairs, where at least one gene was not present in 'x'... Done!
[cyclone] Removed 2836 marker pairs. 26545 marker pairs remaining.
[cyclone] Calculating scores and predicting cell cycle phase... Done!
[cyclone] Calculated scores and prediction (phase: count): S: 99, G2M: 4, G1: 144


In [23]:
prediction_table = helper.get_prediction_table(prediction)
helper.DataTable(prediction_table)

,G1,G2M,S,G1_norm,G2M_norm,S_norm,prediction
sample,,,,,,,
G2_Exp1.059,0.480,0.011,0.396,0.541150,0.012401,0.446449,S
G2_Exp1.069,0.062,0.504,0.053,0.100162,0.814216,0.085622,G2M
G2_Exp1.075,0.004,0.258,0.633,0.004469,0.288268,0.707263,S
G2_Exp1.063,0.043,0.426,0.136,0.071074,0.704132,0.224793,S
G2_Exp1.029,0.281,0.051,0.248,0.484483,0.087931,0.427586,S
G2_Exp1.076,0.042,0.414,0.219,0.062222,0.613333,0.324444,S
G2_Exp1.013,0.367,0.032,0.416,0.450307,0.039264,0.510429,S
G2_Exp1.037,0.835,0.003,0.235,0.778192,0.002796,0.219012,G1
G2_Exp1.057,0.646,0.000,0.233,0.734926,0.000000,0.265074,G1


In [24]:
helper.plot_prediction(
    prediction_table.loc[:, "G1"], 
    prediction_table.loc[:, "S"], 
    prediction_table.loc[:, "G2M"], 
    samples=list(prediction_table.index),
    t="scatter", title="Phase assignment"
)

{'data': [{'marker': {'color': 'black', 'size': 10, 'symbol': 'circle'},
   'mode': 'markers+text',
   'name': 'Sample0',
   'text': ['G2_Exp1.059',
    'G2_Exp1.069',
    'G2_Exp1.075',
    'G2_Exp1.063',
    'G2_Exp1.029',
    'G2_Exp1.076',
    'G2_Exp1.013',
    'G2_Exp1.037',
    'G2_Exp1.057',
    'G2_Exp1.018',
    'G2_Exp1.015',
    'G2_Exp1.019',
    'G2_Exp1.050',
    'G2_Exp1.004',
    'G2_Exp1.061',
    'G2_Exp1.042',
    'G2_Exp1.060',
    'G2_Exp1.058',
    'G2_Exp1.065',
    'G2_Exp1.002',
    'G2_Exp1.044',
    'G2_Exp1.051',
    'G2_Exp1.073',
    'G2_Exp1.030',
    'G2_Exp1.028',
    'G2_Exp1.022',
    'G2_Exp1.034',
    'G2_Exp1.017',
    'G2_Exp1.047',
    'G2_Exp1.072',
    'G2_Exp1.074',
    'G2_Exp1.054',
    'G2_Exp1.024',
    'G2_Exp1.032',
    'G2_Exp1.020',
    'G2_Exp1.064',
    'G2_Exp1.045',
    'G2_Exp1.038',
    'G2_Exp1.001',
    'G2_Exp1.049',
    'G2_Exp1.031',
    'G2_Exp1.039',
    'G2_Exp1.070',
    'G2_Exp1.007',
    'G2_Exp1.021',
    'G2_Exp1.03

In [25]:
p = ["G1","S","G2M"]
label = [] 

for c in gencounts_oscope_sorted.columns:
    if "G1_" in c:
        label.append("G1")
    elif "S_" in c:
        label.append("S")
    elif "G2_" in c:
        label.append("G2M")

In [26]:
evaluation = helper.evaluate_prediction(prediction_table, label)

iplot(helper.plot_evaluation(*evaluation, average=True, xaxislbl=["G1","S","G2M"], title="Prediction Scores per Phase for unnormalized gene counts"), image = 'svg')

F1 Score: G1: 0.4936170212765958, S: 0.13407821229050276, G2M: 0.1
Reacall: G1: 0.6373626373626373, S: 0.15, G2M: 0.05263157894736842 
Precision: G1: 0.4027777777777778, S: 0.12121212121212122, G2M: 1.0 


In [27]:
x = gencounts_oscope.T.values

X_std = QuantileTransformer().fit_transform(x.astype(float))

gencounts_oscope_normalized = pandas.DataFrame(X_std.T, index=gencounts_oscope.index, columns=gencounts_oscope.columns)
gencounts_oscope_normalized_sorted = gencounts_oscope_normalized.iloc[:,
                              [gencounts_oscope_normalized.columns.get_loc(c) for c in gencounts_oscope_normalized.columns if
                              "G1_" in c or "G2_" in c or "S_" in c]]
gencounts_oscope_normalized_sorted.head(10)

,G2_Exp1.059,G2_Exp1.069,G2_Exp1.075,G2_Exp1.063,G2_Exp1.029,G2_Exp1.076,G2_Exp1.013,G2_Exp1.037,G2_Exp1.057,G2_Exp1.018,...,G1_Exp1.008,G1_Exp1.055,G1_Exp1.050,G1_Exp1.076,G1_Exp1.011,G1_Exp1.063,G1_Exp1.083,G1_Exp1.030,G1_Exp1.018,G1_Exp1.046
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
MKL2,5.948365e-01,4.251952e-01,2.353241e-01,4.506200e-01,3.248365e-01,8.494862e-01,6.035429e-01,1.000000e-07,5.642660e-01,9.781442e-01,...,3.512828e-01,7.757691e-01,7.407407e-01,2.723152e-01,2.772233e-01,1.000000e-07,6.386019e-01,4.684572e-01,8.689220e-01,8.305803e-01
CD109,1.003050e-01,3.201135e-01,7.909013e-01,3.080467e-02,8.501255e-02,5.030089e-01,7.447545e-01,9.152499e-01,5.667066e-02,3.573420e-01,...,2.592593e-01,3.856201e-01,2.323796e-01,4.025300e-01,3.136761e-01,1.264310e-01,8.998772e-01,1.290721e-01,5.925926e-01,3.766069e-01
ABTB1,9.020715e-01,1.000000e-07,1.000000e-07,9.211207e-01,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07
MAST2,4.792734e-01,4.836901e-01,8.627706e-01,1.000000e-07,7.818149e-01,1.000000e-07,8.561407e-01,1.000000e-07,1.000000e-07,9.150147e-01,...,5.272325e-01,7.318324e-01,1.000000e-07,8.454940e-01,7.754613e-01,1.000000e-07,5.014795e-01,1.000000e-07,6.666667e-01,8.039964e-01
KAT5,5.337311e-01,4.465861e-01,1.000000e-07,9.829656e-01,1.000000e-07,1.000000e-07,7.804196e-01,5.096150e-01,9.759950e-01,1.000000e-07,...,1.000000e-07,8.538740e-01,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,4.663023e-01,7.668570e-01,5.425416e-01,1.000000e-07
WWC2,1.460928e-01,8.388357e-01,7.306529e-01,1.270348e-01,5.951589e-01,5.510902e-01,8.825024e-01,3.772122e-01,5.226564e-01,8.779768e-01,...,2.723594e-01,3.469510e-01,8.539372e-01,6.712327e-01,9.738533e-01,8.583905e-01,1.571370e-01,5.029423e-01,6.746602e-02,1.111111e-01
CD163,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07
MYL2,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07
UBE2Z,9.478934e-01,6.929424e-01,1.960402e-01,1.851852e-01,4.357608e-01,5.512878e-01,9.197910e-01,6.493849e-01,6.947714e-01,2.418189e-01,...,6.818743e-01,4.377827e-01,9.805805e-01,1.266152e-01,4.770790e-01,6.317808e-02,1.021025e-01,9.890268e-01,1.000000e-07,1.048298e-01


In [28]:
prediction = pairs.cyclone(gencounts_oscope_normalized_sorted, mm_marker_pairs, verbose=True)

[__set_matrix] Original Matrix 'x' has shape 19084 x 247
[__set_matrix] Matrix truncation done. Working with 19084 genes for 247 samples.
[cyclone] Preparing marker pairs, where at least one gene was not present in 'x'... Done!
[cyclone] Removed 2836 marker pairs. 26545 marker pairs remaining.
[cyclone] Calculating scores and predicting cell cycle phase... Done!
[cyclone] Calculated scores and prediction (phase: count): G2M: 94, S: 29, G1: 124


In [29]:
prediction_table = helper.get_prediction_table(prediction)
helper.DataTable(prediction_table)

,G1,G2M,S,G1_norm,G2M_norm,S_norm,prediction
sample,,,,,,,
G2_Exp1.059,0.185,0.705,0.508,0.132332,0.504292,0.363376,G2M
G2_Exp1.069,0.003,1.000,0.174,0.002549,0.849618,0.147833,G2M
G2_Exp1.075,0.000,0.995,0.921,0.000000,0.519311,0.480689,G2M
G2_Exp1.063,0.022,0.989,0.094,0.019910,0.895023,0.085068,G2M
G2_Exp1.029,0.167,0.914,0.388,0.113683,0.622192,0.264125,G2M
G2_Exp1.076,0.010,1.000,0.263,0.007855,0.785546,0.206599,G2M
G2_Exp1.013,0.446,0.473,0.455,0.324600,0.344250,0.331150,S
G2_Exp1.037,0.866,0.194,0.593,0.523896,0.117362,0.358742,G1
G2_Exp1.057,0.693,0.017,0.710,0.488028,0.011972,0.500000,G1


In [30]:
helper.plot_prediction(
    prediction_table.loc[:, "G1"], 
    prediction_table.loc[:, "S"], 
    prediction_table.loc[:, "G2M"], 
    samples=list(prediction_table.index),
    t="pie", title="Phase assignment", width=600, height=600
)

{'data': [{'marker': {'color': 'black', 'size': 10, 'symbol': 'circle'},
   'mode': 'markers+text',
   'name': 'Sample0',
   'text': [' 0',
    ' 1',
    ' 2',
    ' 3',
    ' 4',
    ' 5',
    ' 6',
    ' 7',
    ' 8',
    ' 9',
    ' 10',
    ' 11',
    ' 12',
    ' 13',
    ' 14',
    ' 15',
    ' 16',
    ' 17',
    ' 18',
    ' 19',
    ' 20',
    ' 21',
    ' 22',
    ' 23',
    ' 24',
    ' 25',
    ' 26',
    ' 27',
    ' 28',
    ' 29',
    ' 30',
    ' 31',
    ' 32',
    ' 33',
    ' 34',
    ' 35',
    ' 36',
    ' 37',
    ' 38',
    ' 39',
    ' 40',
    ' 41',
    ' 42',
    ' 43',
    ' 44',
    ' 45',
    ' 46',
    ' 47',
    ' 48',
    ' 49',
    ' 50',
    ' 51',
    ' 52',
    ' 53',
    ' 54',
    ' 55',
    ' 56',
    ' 57',
    ' 58',
    ' 59',
    ' 60',
    ' 61',
    ' 62',
    ' 63',
    ' 64',
    ' 65',
    ' 66',
    ' 67',
    ' 68',
    ' 69',
    ' 70',
    ' 71',
    ' 72',
    ' 73',
    ' 74',
    ' 75',
    ' 76',
    ' 77',
    ' 78',
    ' 79',


In [31]:
evaluation = helper.evaluate_prediction(prediction_table, label=label)

iplot(helper.plot_evaluation(*evaluation, average=True, xaxislbl=["G1","S","G2M"], title="Prediction Scores per Phase for unnormalized gene counts"), image='svg')

F1 Score: G1: 0.4837209302325581, S: 0.20183486238532108, G2M: 0.6705882352941177
Reacall: G1: 0.5714285714285714, S: 0.1375, G2M: 0.75 
Precision: G1: 0.41935483870967744, S: 0.3793103448275862, G2M: 0.6063829787234043 
